# Redes Neuronales - TP2
## Ej 5

Siguiendo el trabajo de Hinton y Salakhutdinov (2006), entrene una máquina restringida
de Boltzmann con imágenes de la base de datos MNIST. Muestre el error de
recontruccion durante el entrenamiento, y ejemplos de cada uno de los dígitos
reconstruidos.




La MRB se puede pensar como una red feedfoward de 4 capas, en vez de pensarla como una de 2 capas con pesos simétricos. $v$ es la capa visible y $h$ es la oculta, y tenemos un $\hat{v}$ y $\hat{h}$, que son las estimaciones. 

Las fórmulas que hay que seguir son las siguientes:

$$
m_i = \text{pixel}_i
$$

$$
v_i \sim  \mathcal{N}(m_i , 1)
$$

$$
P_r (j) = g(\sum _i w_{ij} v_i + b_j) \quad \quad g (x) = \frac{1}{1+e^{-x}}
$$

$$
h(j) = 
\begin{cases}
1 \quad \text{con probabilidad} \quad P_r 
\\
0 \quad  \text{con probabilidad}\quad 1-P_r 
\end{cases}
$$

$$
\text{guardar} \quad (v_i , h_j)_{data}
$$

Luego se repite pero para obtener las estimaciones

$$
m_i = \sum _j w_{ij} v_i + b_i
$$

$$
\hat{v}_i \sim  \mathcal{N}(m_i , 1)
$$

$$
P_r (j) = g(\sum _i w_{ij} v_i + b_j) \quad \quad g (x) = \frac{1}{1+e^{-x}}
$$

$$
\hat{h}(j) = 
\begin{cases}
1 \quad \text{con probabilidad} \quad P_r 
\\
0 \quad  \text{con probabilidad}\quad 1-P_r 
\end{cases}
$$

$$
\text{guardar} \quad (v_i , h_j)_{reconstruccion}
$$

Al final se hace

$$
\Delta w_{ij}= \eta ( \left\langle v_i h_j \right\rangle - \left\langle \hat{v_i} \hat{h_j} \right\rangle)
$$

$$
b_i= \eta ( \left\langle v_i \right\rangle - \left\langle \hat{v_i} \right\rangle)
$$

$$
\Delta w_{ij}= \eta ( \left\langle v_i h_j \right\rangle - \left\langle \hat{v_i} \hat{h_j} \right\rangle)
$$
$$
b_j= \eta ( \left\langle  h_j \right\rangle - \left\langle  \hat{h_j} \right\rangle)
$$

In [1]:
from keras.datasets import mnist
import numpy as np
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [ ]:


def _sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


class RBM:
    """
    RBM con visibles gaussianas (sigma=1) y ocultas binarias.

    He separado los pasos en métodos pequeños para que puedas completarlos o
    modificar su comportamiento más fácilmente:
      - v_to_h(v, sample=True): cálculo de P(h|v) y opcionalmente muestreo
      - h_to_v(h, sample=True): cálculo de la media de v|h y opcionalmente muestreo
      - compute_gradients(v0, h0_prob, v1, h1_prob): calcular dW, db, dc (por defecto CD-1)
      - train_loop(data, ...): superloop que itera por épocas/batches y para cuando converge

    Las implementaciones por defecto realizan CD-1; puedes reimplementar cualquiera
    de los métodos si prefieres otro comportamiento.
    """

    def __init__(self, n_visible, n_hidden, lr=0.1, rng=None):
        self.n_visible = int(n_visible)
        self.n_hidden = int(n_hidden)
        self.lr = float(lr)
        self.rng = np.random.RandomState(None) if rng is None else rng

        # pesos: forma (n_visible, n_hidden)
        self.W = 0.01 * self.rng.randn(self.n_visible, self.n_hidden).astype(np.float32)
        # sesgos visibles y ocultos
        self.b = np.zeros(self.n_visible, dtype=np.float32)
        self.c = np.zeros(self.n_hidden, dtype=np.float32)

    def v_to_h(self, pixeles, sample=True):
        """Paso v -> h.
        Entrada:
          v: array (batch, n_visible)
          sample: si True, devuelve también muestras binarias de h; si False solo probs
        Salida:
          h_prob: P(h=1|v) (batch, n_hidden)
          h_samp: muestras binarias (batch, n_hidden) o None si sample=False

        Nota: por defecto implementado como en las fórmulas: P(h_j=1|v)=sigm(v W + c)
        """
        m = np.asarray(pixeles, dtype=np.float32)  # media = pixel_i
        # sample visibles gaussianos con varianza 1
        v = m + self.rng.randn(*m.shape).astype(np.float32) # la gaussiana tiene varianza 1 y media m. 

        P_r = np.dot(v, self.W) + self.c  # es la activación de la capa de ocultas, pero las usamos como probabilidades para la bernoulli pq son binarias
        h_prob = _sigmoid(P_r)
        h_samp = None
        if sample: # esto es para que genere las muestras binarias de h, que al final las queremos para entrenar
            h_samp = (self.rng.rand(*h_prob.shape) < h_prob).astype(np.float32)
        return h_prob, h_samp

    def h_to_v(self, h, sample=True):
        """Paso h -> v.
        Entrada:
          h: array (batch, n_hidden) -- puede ser probabilidades o muestras
          sample: si True, devuelve muestras gaussianas v ~ N(mean, 1); si False solo la media
        Salida:
          v_mean: media de la distribución (batch, n_visible)
          v_samp: muestras (batch, n_visible) o None si sample=False

        Nota: por defecto asumimos var=1: mean = h W^T + b
        """
        h = np.asarray(h, dtype=np.float32)
        m = np.dot(h, self.W.T) + self.b
        v = m + self.rng.randn(*m.shape).astype(np.float32) # esto es para muestrear v con varianza 1 y media m
        v_samp = None
        if sample: # esto es para que genere las muestras gaussianas de v, que al final las queremos para entrenar
            v_samp = v + self.rng.randn(*v.shape).astype(np.float32)
        return v, v_samp

    def gradiente(self, v0, h0_samp, v1, h1_samp):
        """Calcula los gradientes dW, db, dc dados los estadísticos positivos (v0,h0_prob)
        y negativos (v1, h1_prob).

        Por defecto aplica la regla CD-1 mostrada en el notebook:
          dW = lr * ( <v h>_data - <v h>_recon )
          db = lr * ( <v>_data - <v>_recon )
          dc = lr * ( <h>_data - <h>_recon )

        Devuelve: dW, db, dc (ya escalados por lr)
        """
        # Asegurar arrays y dimensiones
        v0 = np.asarray(v0, dtype=np.float32)
        v1 = np.asarray(v1, dtype=np.float32)
        h0_samp = np.asarray(h0_samp, dtype=np.float32)
        h1_samp = np.asarray(h1_samp, dtype=np.float32)

        batch = float(v0.shape[0]) # la dimensión del batch
        pos_assoc = np.dot(v0.T, h0_samp) / batch # <v h>_data
        neg_assoc = np.dot(v1.T, h1_samp) / batch # <v h>_recon

    # es practicamente lo que está en als ecuaciones

        dW = self.lr * (pos_assoc - neg_assoc)
        db = self.lr * (v0.mean(axis=0) - v1.mean(axis=0))
        dc = self.lr * (h0_samp.mean(axis=0) - h1_samp.mean(axis=0))
        return dW, db, dc

    def reconstruct(self, v):
        """Reconstrucción determinística: v -> h_prob -> v_mean."""
        h_prob, _ = self.v_to_h(v, sample=False)
        v_mean, _ = self.h_to_v(h_prob, sample=False)
        return v_mean

    def train_loop(self, data, max_epochs=50, batch_size=64, tol=1e-4, verbose=True):
        """Superloop de entrenamiento que itera por épocas y batches.

        Logística:
          - data: array (N, n_visible) o iterador/ генератор de minibatches
          - max_epochs: número máximo de épocas
          - batch_size: tamaño de minibatch si `data` es un array
          - tol: criterio de parada si la variación del error por época es menor que tol

        Comportamiento: por cada minibatch realiza los pasos v->h, h->v, calcula gradientes
        usando `compute_gradients` y actualiza parámetros. Devuelve lista de errores por época.
        """
        # Aceptar `data` como array numpy
        X = np.asarray(data, dtype=np.float32)
        N = X.shape[0]
        n_batches = max(1, N // batch_size)

        epoch_errors = []
        prev_epoch_err = None

        for epoch in range(1, max_epochs + 1):
            perm = self.rng.permutation(N)
            epoch_err = 0.0
            for b in range(n_batches):
                idx = perm[b * batch_size:(b + 1) * batch_size]
                v0 = X[idx]

                # Paso 1: v -> h (obtener probabilidades y muestras)
                h0_prob, h0_samp = self.v_to_h(v0)

                # v0 con h0 sería el vector de data

                # Paso 2: h -> v (reconstrucción)
                v1_mean, v1_samp = self.h_to_v(h0_samp)

                # Paso 3: v(recon) -> h (prob negative)
                h1_prob, h1_samp = self.v_to_h(v1_samp)

                # Paso 4: calcular gradientes
                dW, db, dc = self.gradiente(v0, h1_samp, v1_samp, h1_samp)  # acá uso las samples pq la capa intermedia es binaria

                # Aplicar actualizaciones
                self.W += dW
                self.b += db
                self.c += dc

                # Acumular error del batch (MSE entre v0 y v1_mean)
                batch_err = np.mean((v0 - v1_mean) ** 2)
                epoch_err += batch_err

            epoch_err /= float(n_batches)
            epoch_errors.append(epoch_err)

            if verbose:
                print(f"Epoch {epoch:3d}/{max_epochs} - recon_error: {epoch_err:.6f}")

            # Criterio de convergencia: cambio pequeño en el error promedio por época
            if prev_epoch_err is not None and abs(prev_epoch_err - epoch_err) < tol:
                if verbose:
                    print("Converged (tol reached). Stopping training.")
                break
            prev_epoch_err = epoch_err

        return epoch_errors


# Ejemplo de uso rápido (prueba de humo)
if __name__ == "__main__":
    rbm = RBM(28 * 28, 128, lr=0.01)
    X = np.random.randn(500, 28 * 28).astype(np.float32)
    errs = rbm.train_loop(X, max_epochs=3, batch_size=128, verbose=True)
    print('Finished, epoch errors:', errs)
